**IMPORTS:**

In [ ]:
import mtcnn
#from facenet_pytorch import MTCNN
import torch
from torch.autograd import Variable
from torchvision.transforms import transforms
from imutils.video import VideoStream
import cv2
from PIL import Image
import matplotlib.pyplot as plt

**GPU or CPU:**

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


**INSANTIATION:**

Object Detection Model - MTCNN

Mask Detection Model - trained pytorch_resnet-34

In [ ]:
Detectionmodel=mtcnn.MTCNN()

Basemodel=torch.load('model.pth', map_location=torch.device('cpu'))
Basemodel.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

**Pred and tsfm methods:**

For Base model

In [ ]:
classes = ['With_Mask', 'Without_Mask']

tsfm = transforms.Compose([
    #transforms.ToPILImage(),
    #transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # mean across each RGB channel
                         [0.229, 0.224, 0.225])
])

def pred(img, tsfm):
    tensor_img = tsfm(img)

    tensor_img = torch.unsqueeze(tensor_img,0) #only for single image pred rather than batch pred
    

    imginput = Variable(tensor_img)

    prediction = Basemodel(imginput)
    index = prediction.data.numpy().argmax()
    #print(index)
    output = classes[index]

    return output

**VIDEO CAPTURE & REALTIME PREDICTION:**

With Bounding Box

In [ ]:
cap=cv2.VideoCapture(0)
while cap.isOpened():
    
    _, frame=cap.read()
    img=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces=Detectionmodel.detect_faces(img)
    #print(faces)
    
    #for mtcnn
    for result in faces:
        x,y,w,h=result['box']
    
        roi_color = frame[y:y+h, x:x+w]
        roi_resize = cv2.resize(roi_color, (224, 224))
        
        output=pred(roi_resize, tsfm)
        #print(output)
        
        label = "Masked" if output=='With_Mask' else "Not Masked!"
        
        colour = (0, 255, 0) if label == "Masked" else (0, 0, 255)
        
        cv2.putText(frame, label, (x,y-10),cv2.FONT_HERSHEY_SIMPLEX, 0.7, colour, 2)
        
        cv2.rectangle(frame,(x,y),(x+w, y+h), colour,4)
            
    #for facenet-pytorch
    '''
    #print(boxes)
    #if(boxes is not None):
    if(cropped_tensor is not None):   
        boxes,_=Detectionmodel.detect(img)
        
        for i, prob in enumerate(probability):
            if prob>0.90:
                box=boxes[i]
                #print(box)
        #print(boxes)
        #for box in boxes:
        
                #roi=frame[int(box[1]):int(box[3]) , int(box[0]):int(box[2])]
                
                #break
                
                output=pred(cropped_tensor[i],tsfm)
                print(output)
                
            
                frame=cv2.rectangle(frame, (int(box[0]),int(box[1])) , (int(box[2]),int(box[3])), (0,0,255), 6)
     '''   
    cv2.imshow("feed",frame)

    if cv2.waitKey(1) & 0xFF == ord('f'):
        break
        
cap.release()
cv2.destroyAllWindows()
    